In [1]:

!pip install torch
!pip install sentence_transformers
import json
from sentence_transformers import SentenceTransformer
import numpy as np
import os


You are using pip version 19.0.3, however version 21.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.3, however version 21.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:


#Init the BERT model to compute sentence embedding
#deberta-v2-xlarge-mnli 
model_name = 'paraphrase-distilroberta-base-v1' #distilbert-base-nli-stsb-mean-tokens'

model = SentenceTransformer(model_name)



I0302 16:14:26.752358 4596407808 SentenceTransformer.py:39] Load pretrained SentenceTransformer: paraphrase-distilroberta-base-v1
I0302 16:14:26.753896 4596407808 SentenceTransformer.py:43] Did not find folder paraphrase-distilroberta-base-v1
I0302 16:14:26.755661 4596407808 SentenceTransformer.py:49] Try to download model from server: https://sbert.net/models/paraphrase-distilroberta-base-v1.zip
I0302 16:14:26.757628 4596407808 SentenceTransformer.py:100] Load SentenceTransformer from folder: /Users/ssedai/.cache/torch/sentence_transformers/sbert.net_models_paraphrase-distilroberta-base-v1
I0302 16:14:28.500162 4596407808 SentenceTransformer.py:124] Use pytorch device: cpu


In [4]:
#download the data
if(not os.path.exists('oos-eval')):
  !git clone https://github.com/clinc/oos-eval.git

In [5]:
def create_dataset(data_split, label_maps=None):
    """
    Extracts features from sentences 
    returns features, class labels and a tuple consisting of label map dict and its inverse
    """
    labels = []
    sentences=[]
    for d in data_split:
        sentences.append(d[0])
        labels.append(d[1])
    if(label_maps is None):
        unique_labels = list(set(labels))
        label_map = { ul:i for  i, ul in enumerate(unique_labels)}
        label_map_inverse = { i:ul for  i, ul in enumerate(unique_labels)}
    else:
        label_map, label_map_inverse = label_maps
    
    labels_class=[label_map[l] for l in labels]
    sentence_embeddings = model.encode(sentences)
    return sentence_embeddings, labels_class, (label_map,label_map_inverse)

In [6]:
#open imbalanced data
with open('oos-eval/data/data_imbalanced.json') as json_file:
    data = json.load(json_file)
    print(data.keys())

    train =  data['train']
    val =  data['val']
    test =  data['test']
             
    oos_train =  data['oos_train']
    oos_val =  data['oos_val']
    oos_test =  data['oos_test']

dict_keys(['oos_val', 'val', 'train', 'oos_test', 'test', 'oos_train'])


In [7]:
#Extract features/labels for inscope set
train_features, train_labels, label_maps = create_dataset(train, label_maps=None)
val_features, val_labels, _ = create_dataset(val, label_maps=label_maps)
test_features, test_labels, _ = create_dataset(test, label_maps=label_maps)

In [8]:
#save features/labels for inscope set
np.savez('train_val_test_'+model_name+'.npz',train_features=train_features,
                              train_labels = train_labels, 
                              val_features=val_features, 
                              val_labels=val_labels,
                              test_features=test_features, 
                              test_labels=test_labels,
                              label_maps=label_maps, allow_pickle=True)

In [9]:
#plot label histogram
import numpy as np
import matplotlib.pyplot as plt
plt.hist(train_labels, bins=50)
plt.show()

<Figure size 640x480 with 1 Axes>

In [10]:
#Extract features/labels for oos set
train_features_oos, train_labels_oos, label_maps_oos = create_dataset(oos_train, label_maps=None)
val_features_oos, val_labels_oos, _ = create_dataset(oos_val, label_maps=label_maps_oos)
test_features_oos, test_labels_oos, _ = create_dataset(oos_test, label_maps=label_maps_oos)

In [11]:
#save features/labels for oos set
np.savez('train_val_test_oos_'+model_name+'.npz',
                              train_features_oos=train_features_oos,
                              train_labels_oos = train_labels_oos, 
                              val_features_oos=val_features_oos, 
                              val_labels_oos=val_labels_oos,
                              test_features_oos=test_features_oos, 
                              test_labels_oos=test_labels_oos,
                              label_maps_oos=label_maps_oos, allow_pickle=True)

In [12]:
label_maps_oos

({'oos': 0}, {0: 'oos'})

In [13]:
for k in  data.keys():
    print(k, len(data[k]))

oos_val 100
val 3000
train 10525
oos_test 1000
test 4500
oos_train 100
